 So the overall plan is something like:

 We want to be able to train a model, where the neuron activations have a particular signal, which guides them to activate according to a particular feature.
 There are two main ways that we can train this feature - we either give a training signal which is purely the signal

 Benefits of pure training:
 Confidence that the network is intending to mean what we want it to mean

 Benefits of mixed training:
 Allows the use of the downstream signal to disambiguate between possible generalizations of the concept that we want.
 This was the initial motivation in the ELK lens - we use the downstream training signal to push the question answerer towards the direct reporter, rather than the human simulator, because the former is more useful on the downstream tasks.

 This should also show up in working better with sparse labels, which we expect to be very useful since manual labels are very expensive (though automating the labelling will be necessary and might make this less useful). 


 My hope was that if we trained separate Joint models, and then 'crossed the wires', so the image -> concept part of model 1 was passing its outputs to the concept -> class function of model 2 50% of the time, while 50% it still passed it to model 1, and these concept -> image model were frozen, the model would be forced to switch to a concept representation that matched to only that which the models had in common.

 We can see this in the clustering of the results here. Looks like there's 

 We can show this 

 Aside: Can we look at this through the lens of single-basin theory and see which attributes match up well to something that arises in the non-attribute loss case.?



 We started off by doing toy models in simple domains. The original idea came from autoencoders, where we would encode

 We see this in our early experiments using autoencoders, though it would be good to be back try to demonstrate this in a feed-forward setting - I ran some basic experiments on this but it wasn't working well - this might be a serious issue but it's not the issue faced by the experiments at the moment. Hypothesis - it depends on the relative complexity of the pre and post true functions.

I moved on to getting this system working with the CUB dataset. This is a dataset which contains >10K images of birds, which are each labelled as being one of 200 species of bird. They are also given binary labels for 312 features which describe the birds' appearances. These categories are divided into 28 categories, such as beak-shape, and only one of these categories are true for any image.

Although one would expect that the actual characteristics of the bird are the same for each bird, the labels differ significantly between birds, and labels are also often marked not-visible. As a result of this, most papers which use the CUB dataset make the task  easier by reducing the number of categories to those which are true in at least 10 cases, which is around 110 (109 in my own experiments, 112 in the original). They also make it much easier by doing a majority-voting transform, by which the labels for each image are replaced by the most common category forbirds of that species, within the test set.

This second transform not only makes the task much easier but changes the entire concept of building a concept model - because to predict the way to predict the features is in fact to predict the class and then use this to deterministically predict all of the features. It is unsurprising that the models which do this perform well, but it is not the task that we are interested in, so I stopped using this transform. This also has the benefit that, being a much harder task, there's a big gap between the performance of a sequential model (about 30% top-1 on test set??) and of the joint models, (about 70% top-1 on the test set), and in the opposite direction on the performance with full intervention - only about 18% with joint, 30%  with sequential, and 50% on independent models. This gives a solid area 

The main thing we want to see is what happens when we take two models which have been trained separately, and train them such that we've crossed the wires.

We have two main metrics - the performance on the test set, and the performance on the test set when the estimated concepts have been fully replaced by the true values - this second test is the best evidence we have of the fidelity of the concept model.

I trained a series of joint models to the point at which their validation loss plateaus, and then merged them into a joint model, which trains them for 20 epochs separately, and then 50 epochs while shuffling the wires, while freezing the premodels, so that the concept outputs don't change after epoch 20. The results are shown below.

What we see is that the train performance, which continues to rise until the 20th epoch, then drops and plateaus, but crucially the validation loss stays the same. This indicates that the information available to the post-model which is communicating consistent information rather than overfitting signal, transfers across the two different models. This is unfortunate as it suggests that there
width="70%"
<div>
<img src="../images/joint2multi_trainvalloss.png", width="40%">
</div>

To test this hypothesis further, I train two multimodels without ever shuffling, and continuously keep track of the validation loss